<a href="https://colab.research.google.com/github/licyk/sd-webui-for-colab/blob/main/stable_diffusion_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StableDiffusion CoLab版**

 **🚨注意：CoLab官方开始禁止CoLab免费用户使用stable diffusion进行跑图，但stable diffusion的模型训练(如Dreambooth，LoRA)依然可以正常进行**

 **Dreambooth，LoRA训练脚本推荐：**

 **[Training Colabs(来自camenduru)](https://github.com/camenduru/stable-diffusion-webui-colab/tree/training)**

 **[Dreambooth autodl(来自秋葉aaaki)](https://github.com/Akegarasu/dreambooth-autodl)**

 **[Lora-scripts(来自秋葉aaaki)](https://github.com/Akegarasu/lora-scripts)**

 **[Colab_Lora_train(来自WSH032)](https://colab.research.google.com/github/WSH032/lora-scripts/blob/main/Colab_Lora_train.ipynb)**

 脚本来自https://github.com/camenduru/stable-diffusion-webui-colab

 对脚本内容进行分段

**使用注意：**

请确保CoLab会话类型为GPU运行时

若不是，则在**“代码执行工具--->更改运行时类型--->硬件加速器”**更改为GPU

In [ ]:
#@title 检查GPU
import os, subprocess


try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    gpuinfo = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if 'M4000' in gpuinfo:
        print("注意: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
    else:
        print("当前使用："+gpuinfo.strip())
except:
    print("没有GPU可使用，请检查你的运行时类型")
    exit()

In [ ]:
#@title 初始化环境 
!echo "初始化环境中..."
%cd /content

import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

#!pip install torch==2.00+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!apt -y install -qq aria2
#!pip install -q xformers==0.0.17
#!pip install -q triton==2.0.0
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U

#下载stable diffusion
#!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
#!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py

%cd /content/stable-diffusion-webui
!git reset --hard

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt

!echo "初始化完成"

In [ ]:
#@title 下载模型
!echo "下载模型中..."
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/ChilloutMix-ni-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o ChilloutMix-ni-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/licyk/sd-vae/resolve/main/pastel-mix-vae.pt -d /content/stable-diffusion-webui/models/VAE -o pastel-mix-vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0-pruned-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.0-pruned-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -d /content/stable-diffusion-webui/models/VAE -o anything-v4.0.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/licyk/sd-vae/resolve/main/orangemix.vae.pt -d /content/stable-diffusion-webui/models/VAE -o orangemix.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/licyk/sd-model/resolve/main/AOM3_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3_orangemixs.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/licyk/sd-model/resolve/main/cetusMix_cetusVersion3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o cetusMix_cetusVersion3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/licyk/sd-model/resolve/main/ER-need-orangemix.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ER-need-orangemix.ckpt
!echo "下载完成"

In [ ]:
#@title 下载插件
!echo "下载插件中..."
#sd-webui-additional-networks
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
#a1111-sd-webui-tagcomplete
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
#multidiffusion-upscaler-for-automatic1111
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/stable-diffusion-webui/extensions/multidiffusion-upscaler-for-automatic1111
#sd-dynamic-thresholding
!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding /content/stable-diffusion-webui/extensions/sd-dynamic-thresholding
#sd-webui-cutoff
!git clone https://github.com/hnmr293/sd-webui-cutoff /content/stable-diffusion-webui/extensions/sd-webui-cutoff
#sd-webui-model-converter
!git clone https://github.com/Akegarasu/sd-webui-model-converter /content/stable-diffusion-webui/extensions/sd-webui-model-converter
#sd-webui-supermerger
!git clone https://github.com/hako-mikan/sd-webui-supermerger /content/stable-diffusion-webui/extensions/sd-webui-supermerger
#stable-diffusion-webui-localization-zh_CN
!git clone https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-zh_CN
#stable-diffusion-webui-wd14-tagger
!git clone https://github.com/tsukimiya/stable-diffusion-webui-wd14-tagger /content/stable-diffusion-webui/extensions/stable-diffusion-webui-wd14-tagger
#sd-webui-regional-prompter
!git clone https://github.com/hako-mikan/sd-webui-regional-prompter /content/stable-diffusion-webui/extensions/sd-webui-regional-prompter
#stable-diffusion-webui-baidu-netdisk
!git clone https://github.com/zanllp/stable-diffusion-webui-baidu-netdisk /content/stable-diffusion-webui/extensions/stable-diffusion-webui-baidu-netdisk
#stable-diffusion-webui-anti-burn
!git clone https://github.com/klimaleksus/stable-diffusion-webui-anti-burn /content/stable-diffusion-webui/extensions/stable-diffusion-webui-anti-burn
#loopback_scaler
!git clone https://github.com/Elldreth/loopback_scaler.git /content/stable-diffusion-webui/extensions/loopback_scaler
#latentcoupleregionmapper
!git clone https://github.com/CodeZombie/latentcoupleregionmapper.git /content/stable-diffusion-webui/extensions/latentcoupleregionmapper
#ultimate-upscale-for-automatic1111
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git /content/stable-diffusion-webui/extensions/ultimate-upscale-for-automatic1111
#deforum-for-automatic1111-webui
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!mkdir /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui/models
#stable-diffusion-webui-images-browser
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
#stable-diffusion-webui-huggingface
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
#sd-civitai-browser
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
#sd-webui-additional-networks
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
#openpose-editor
!git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
#sd-webui-depth-lib
!git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/sd-webui-depth-lib
#posex
!git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex
#sd-webui-tunnels
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
#batchlinks-webui
!git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
#stable-diffusion-webui-catppuccin
!git clone https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
#a1111-sd-webui-locon
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon
#stable-diffusion-webui-rembg
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg /content/stable-diffusion-webui/extensions/stable-diffusion-webui-rembg
#stable-diffusion-webui-two-shot
!git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot
#sd-webui-lora-block-weight
!git clone https://github.com/hako-mikan/sd-webui-lora-block-weight /content/stable-diffusion-webui/extensions/sd-webui-lora-block-weight
#sd-face-editor
!git clone https://github.com/ototadana/sd-face-editor /content/stable-diffusion-webui/extensions/sd-face-editor
#controlnet
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
#controlnet插件的相关模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth
!echo "下载完成"

In [ ]:
#@title 下载lora模型
#!apt install --only-upgrade git-lfs -y
!echo "下载lora模型"
!git clone https://huggingface.co/licyk/sd-lora /content/sd-lora
!echo "导入中..."
!mv -f /content/sd-lora/* /content/stable-diffusion-webui/models/Lora
!rm -rf /content/sd-lora
!echo "导入成功"

In [ ]:
#@title 启动stable diffusion
!echo "开始启动stable diffusion"
!echo "出现的蓝色链接为webui地址"
%cd /content/stable-diffusion-webui/
!python launch.py --theme dark --listen --xformers --enable-insecure-extension-access --no-half --no-half-vae  --precision full --gradio-queue --multiple

In [ ]:
#@title 挂载google云端硬盘
from google.colab import drive
drive.mount('/content/drive')
!echo "挂载google driver成功"

In [ ]:
#@title 将生成的图片导出到google云端硬盘
!echo "将会在google云端硬盘生成sdwebui-output文件夹"
!mkdir /content/drive/MyDrive/sdwebui-output
!cp -r /content/stable-diffusion-webui/outputs/* /content/drive/MyDrive/sdwebui-output/
!echo "导出成功"

# 后记：

 stable diffusion版本可自行切换

 stable diffusion在启动好后会在输出项显示webui蓝色链接

 启动stable diffusion后可以在**settings->user interface->Localization** 选择中文并重启ui

 在**settings->user interface->Quicksettings list**中添加**sd_model_checkpoint, sd_vae, CLIP_stop_at_last_layers**后并重启ui(每个参数用英文的逗号隔开)

 stable diffusion 装有civital和hugging face下载插件，可以在上面下载模型

如果webui界面出现错误、无法连接的情况时，可以选择其他的webui链接，或者重新运行**“启动stable diffusion”**

 在使用完脚本后可以运行**“将生成的图片导出到google云端硬盘”**将画好的图保存到google云端硬盘

如果要切换成白色webui界面，可以禁用stable-diffusion-webui-catppuccin插件

 参考：
 
 [(CSDN)Stable Diffusion攻略集](https://blog.csdn.net/qq_56591814/article/details/128385416)
 
 [SD-WEB官方wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki)
